In [ ]:

# 🧠 LSTM Deployment in Colab using Streamlit + pyngrok

# ✅ Step 1: Install required libraries
!pip install streamlit pyngrok pandas numpy matplotlib scikit-learn tensorflow --quiet

# ✅ Step 2: Write the Streamlit app code to a Python file
app_code = """
import streamlit as st
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# Load model
model = load_model("lstm_model_20250625_154404.h5")

def create_sequences(data, window_size=30):
    X = []
    for i in range(window_size, len(data)):
        X.append(data[i-window_size:i])
    return np.array(X)

st.title("🧪 LSTM Sales Forecast - Store 1")

uploaded_file = st.file_uploader("Upload Store 1 Sales CSV", type=["csv"])
if uploaded_file is not None:
    df = pd.read_csv(uploaded_file, dtype={'StateHoliday': str})
    df = df[(df['Store'] == 1) & (df['Open'] == 1) & (df['Sales'] > 0)].copy()
    df = df.sort_values("Date")
    sales = df["Sales"].values.reshape(-1, 1)

    scaler = MinMaxScaler()
    scaled_sales = scaler.fit_transform(sales)
    X = create_sequences(scaled_sales)

    if len(X) > 0:
        y_pred_scaled = model.predict(X)
        y_pred = scaler.inverse_transform(y_pred_scaled)

        df_result = df.iloc[30:].copy()
        df_result["Predicted_Sales"] = y_pred

        st.line_chart(df_result[["Sales", "Predicted_Sales"]])
        st.download_button("📥 Download Predictions", df_result.to_csv(index=False), "predicted_sales.csv")
    else:
        st.warning("❌ Not enough data (need 30+ days of Store 1 sales).")
"""

with open("app.py", "w") as f:
    f.write(app_code)

# ✅ Step 3: Upload your .h5 model file manually in the left pane OR use:
# from google.colab import files; files.upload()

# ✅ Step 4: Run the Streamlit app with ngrok
from pyngrok import ngrok

public_url = ngrok.connect(port=8501)
print("🌐 Public URL:", public_url)

!streamlit run app.py & npx localtunnel --port 8501
